In [ ]:
# How much is wind power at a particular location ?
# Station Name : OAK PARK 
# Wind speed is analysed by hourly basic 

In [4]:
import pandas as pd
import sqlite3


#steps 

# load data 
# from file name 
file_path = 'C:/Users/sarab/Downloads/hly375/hly375/hly375.csv'
data = pd.read_csv(file_path, skiprows=17)  # Skip the first 17 rows of metadata

print(data)





# Rename columns to match the dataset's ddescription 
data.columns = ['date', 'irain', 'rain', 'itemp', 'temp', 'iwetb', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'iwsdp', 'wdsp', 'iwddir', 'wddir']



# Apply column-wise filtering based on indicators
# Rainfall: Keep rows where irain is between 0 and 6, drop rain values where irain == -1
data.loc[data['irain'] == -1, 'rain'] = None


# Temperature: Drop temp values where itemp == 4
data.loc[data['itemp'] == 4, 'temp'] = None

# Wet Bulb: Drop wetb values where iwetb == 4
data.loc[data['iwetb'] == 4, 'wetb'] = None

# Wind Speed: Drop wdsp values where iwsdp == 7
data.loc[data['iwsdp'] == 7, 'wdsp'] = None

# Wind Direction: Drop wddir values where iwddir == 7
data.loc[data['iwddir'] == 7, 'wddir'] = None


# Drop rows with missing values in any critical columns
data_cleaned = data.dropna(subset=['rain', 'temp', 'wetb', 'wdsp', 'wddir'])

# Convert date column to datetime format
data_cleaned['date'] = pd.to_datetime(data_cleaned['date'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['date'])













C:\Users\sarab\AppData\Local\Temp\ipykernel_23440\384118421.py:10: DtypeWarning: Columns (2,4,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, skiprows=17)  # Skip the first 17 rows of metadata


                     date  ind rain  ind.1  temp  ind.2  wetb dewpt vappr  \
0       01-aug-2003 01:00   -1           4            4                     
1       01-aug-2003 02:00   -1           4            4                     
2       01-aug-2003 03:00   -1           4            4                     
3       01-aug-2003 04:00   -1           4            4                     
4       01-aug-2003 05:00   -1           4            4                     
...                   ...  ...  ...    ...   ...    ...   ...   ...   ...   
187027  30-nov-2024 20:00    0  0.5      0  13.4      0  13.0  12.7  14.6   
187028  30-nov-2024 21:00    0  1.5      0  13.3      0  12.9  12.7  14.6   
187029  30-nov-2024 22:00    0  0.2      0  13.4      0  12.8  12.3  14.3   
187030  30-nov-2024 23:00    0  0.3      0  13.1      0  12.4  11.8  13.9   
187031  01-dec-2024 00:00    0  0.0      0  13.0      0  12.5  12.0  14.0   

       rhum     msl  ind.3 wdsp  ind.4 wddir  
0                        7  

C:\Users\sarab\AppData\Local\Temp\ipykernel_23440\384118421.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_cleaned['date'] = pd.to_datetime(data_cleaned['date'], errors='coerce')
C:\Users\sarab\AppData\Local\Temp\ipykernel_23440\384118421.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['date'] = pd.to_datetime(data_cleaned['date'], errors='coerce')


184638